In [8]:
import numpy as np
import pandas as pd

## Simple `groupby` example

In [1]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


Simple `groupby`:

1. Split the DataFrame based on the specified key\
    *Here we split based on the `key` column. We can think of it as the originial bigDataFrame is splited into a collection of small DataFrames, which contains A, B or C.*

2. Apply the desired aggregatio operation\
    *Here we apply simply the `sum()`*
    
3. Combine the results of all DataFrames together

In [3]:
df.groupby('key') # split based on the `key` column

In [4]:
df.groupby('key').sum() # apply sum() and combine the results together

,data
key,
A,3
B,5
C,7


## `DataFrameGroupBy` object

In [5]:
df.groupby('key')

`groupby()` returns a `DataFramGroupBy` object. We can simply treat it as a collection of small `DataFrame`s.

On `DataFramGroupBy` we can do:

- Column indexing (as in `DataFrame`)

In [11]:
df1 = df.copy()
df1['data2'] = np.random.randint(10, size=6)
df1

,key,data,data2
0,A,0,1
1,B,1,1
2,C,2,5
3,A,3,6
4,B,4,8
5,C,5,4


In [12]:
df1.groupby('key')['data2'].sum()

key
A    7
B    9
C    9
Name: data2, dtype: int64

- Iteration over groups

In [16]:
for key, group in df.groupby('key'):
    print(f'key: {key} \n {group} \n')

key: A 
   key  data
0   A     0
3   A     3 

key: B 
   key  data
1   B     1
4   B     4 

key: C 
   key  data
2   C     2
5   C     5 



- Dispatch methods

    Methods of `DataFrame` and `Series` can be passed through and applied to each individual group. The results are then combined within `GroupBy` and returned.

In [20]:
df1.groupby('key')['data2'].describe()

,count,mean,std,min,25%,50%,75%,max
key,,,,,,,,
A,2.0,3.5,3.535534,1.0,2.25,3.5,4.75,6.0
B,2.0,4.5,4.949747,1.0,2.75,4.5,6.25,8.0
C,2.0,4.5,0.707107,4.0,4.25,4.5,4.75,5.0


## Flexible control

`DataFrameGroupBy` objects have `aggregate()`, `filter()`, `transform()`, and `apply()` methods that efficiently implement a variety of useful operations before combining the grouped data.

### Aggregation

`aggregate()`: can take a string, a function, or a list thereof, and compute the aggregates **at once**.

Pandas built-in aggregation:

| Aggregation          | Description                     |
| :------------------- | :------------------------------ |
| `count()`            | Total number of items           |
| `first()`, `last()`  | First and last item             |
| `mean()`, `median()` | Mean and median                 |
| `min()`, `max()`     | Minimum and maximum             |
| `std()`, `var()`     | Standard deviation and variance |
| `mad()`              | Mean absolute deviation         |
| `prod()`             | Product of all items            |
| `sum()`              | Sum of all items                |

In [25]:
df1

,key,data,data2
0,A,0,1
1,B,1,1
2,C,2,5
3,A,3,6
4,B,4,8
5,C,5,4


In [21]:
df1.groupby('key').aggregate(['min', max, np.median])

data            data2           
     min max median   min max median
key                                 
A      0   3    1.5     1   6    3.5
B      1   4    2.5     1   8    4.5
C      2   5    3.5     4   5    4.5

In [26]:
df1.groupby('key').aggregate({'data':['min', max], 'data2': [np.min, np.median, np.mean]})

data     data2            
     min max  amin median mean
key                           
A      0   3     1    3.5  3.5
B      1   4     1    4.5  4.5
C      2   5     4    4.5  4.5

### Filtering

Syntax: `df.groupby(key).filter(filter_criteria)`

E.g.: we might want to keep all groups in which the mean of `data2` column is larger than some critical value:

In [29]:
def filter_func(x):
    return x['data2'].mean() > 4


df1.groupby('key').filter(filter_func)

,key,data,data2
1,B,1,1
2,C,2,5
4,B,4,8
5,C,5,4


### Transformation

Transformation can return some transformed version of the full data to recombine.

Syntax: `df.groupby(key).transform(transform_func)`

In [32]:
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [33]:
df.groupby('key').mean()

,data
key,
A,1.5
B,2.5
C,3.5


In [31]:
df.groupby('key').transform(lambda x: x - x.mean())

,data
0,-1.5
1,-1.5
2,-1.5
3,1.5
4,1.5
5,1.5


### Apply

Apply an arbitrary function to the group results.

E.g.:

In [38]:
def norm(x):
    x['data'] /= (x['data'] + x['data2'])
    return x

In [39]:
df1

,key,data,data2
0,A,0,1
1,B,1,1
2,C,2,5
3,A,3,6
4,B,4,8
5,C,5,4


In [48]:
df1.groupby('key').apply(norm)

,key,data,data2
0,A,0.000000,1
1,B,0.500000,1
2,C,0.285714,5
3,A,0.333333,6
4,B,0.333333,8
5,C,0.555556,4


In [52]:
df1.groupby('key').sum()

,data,data2
key,,
A,3,7
B,5,9
C,7,9


## Specify the split key

- List, array, `Series` or index as grouping keys

- A dictionary or series mapping index to group

In [41]:
df2 = df.set_index('key')
df2

,data
key,
A,0
B,1
C,2
A,3
B,4
C,5


In [42]:
mapping = {
    'A': 'foo',
    'B': 'bar',
    'C': 'foo'
}

df2.groupby(mapping).sum()

,data
bar,5
foo,10


- Pass any Python function that will input the index value and output the group

In [43]:
df2.groupby(str.upper).sum()

,data
A,3
B,5
C,7


- A list of valid keys\
Any of the preceding key choices can be combined to group on a multi-index

In [46]:
df2.groupby([str.upper, mapping]).sum()

,,data
A,foo,3
B,bar,5
C,foo,7
